In [3]:
from jsonformer.pydantic_parser import create_scheme
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer
from typing import Literal
from pydantic import BaseModel

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model and tokenizer...")
model_name = "databricks/dolly-v2-3b"
model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

In [6]:
color_names = Literal[
    "red",
    "green",
    "black"
]

class Color(BaseModel):
    name: color_names
    id: str

class Performance(BaseModel):
    engine: str
    horsepower: float
    topSpeed: float

class Safety(BaseModel):
    airbags: int 
    parkingSensors: bool 
    laneAssist: bool

class Audio(BaseModel):
    brand: str
    speakers: int
    hasBluetooth: bool

class Features(BaseModel):
    audio: Audio
    safety: Safety
    performance: Performance

class Car(BaseModel):
    make: str
    model: str
    year: int 
    colors: list[Color]
    features: Features

class Owner(BaseModel):
    firstName: str
    lastName: str
    age: int

class CarObject(BaseModel):
    car: Car
    owner: Owner

car_object = create_scheme(CarObject)
car_object

{'type': 'object',
 'properties': {'car': {'type': 'object',
   'properties': {'make': {'type': 'string'},
    'model': {'type': 'string'},
    'year': {'type': 'number'},
    'colors': {'type': 'array',
     'items': {'type': 'object',
      'properties': {'name': {'type': 'enum',
        'values': ['red', 'green', 'black']},
       'id': {'type': 'string'}}}},
    'features': {'type': 'object',
     'properties': {'audio': {'type': 'object',
       'properties': {'brand': {'type': 'string'},
        'speakers': {'type': 'number'},
        'hasBluetooth': {'type': 'boolean'}}},
      'safety': {'type': 'object',
       'properties': {'airbags': {'type': 'number'},
        'parkingSensors': {'type': 'boolean'},
        'laneAssist': {'type': 'boolean'}}},
      'performance': {'type': 'object',
       'properties': {'engine': {'type': 'string'},
        'horsepower': {'type': 'number'},
        'topSpeed': {'type': 'number'}}}}}}},
  'owner': {'type': 'object',
   'properties': {'first

In [ ]:
builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car_object,
    prompt="generate an example car",
    max_string_token_length=20,
)

print("Generating...")
output = builder()

highlight_values(output)